In [178]:
import pandas as pd
import numpy as np
import scipy.stats

from scipy.stats import ttest_ind
from scipy.stats import levene

#Load Breast Cancer Dataset
df = pd.read_csv('/Users/maryamazgomi/Documents/ECS171/brca.csv') #Breast Cancer Wisconsin Dataset

print (df.head())

   Unnamed: 0  x.radius_mean  x.texture_mean  x.perimeter_mean  x.area_mean  \
0           1         13.540           14.36             87.46        566.3   
1           2         13.080           15.71             85.63        520.0   
2           3          9.504           12.44             60.34        273.9   
3           4         13.030           18.42             82.61        523.8   
4           5          8.196           16.84             51.71        201.9   

   x.smoothness_mean  x.compactness_mean  x.concavity_mean  \
0            0.09779             0.08129           0.06664   
1            0.10750             0.12700           0.04568   
2            0.10240             0.06492           0.02956   
3            0.08983             0.03766           0.02562   
4            0.08600             0.05943           0.01588   

   x.concave_pts_mean  x.symmetry_mean  ...  x.texture_worst  \
0            0.047810           0.1885  ...            19.26   
1            0.031100   

In [188]:
#Check if 'y' exists
if 'y' in df.columns:
    df['Diagnosis'] = df['y'].map({'B': 0, 'M': 1})
    df.drop(columns=['Unnamed: 0', 'y'], inplace=True, errors='ignore')  # Drop unnecessary columns
else:
    print("Warning: Column 'y' not found. Available columns:", df.columns)

# Separate benign and malignant cases
if 'Diagnosis' in df.columns:
    benign = df[df['Diagnosis'] == 0]
    malignant = df[df['Diagnosis'] == 1]
else:
    raise KeyError("Diagnosis column not found after renaming. Check dataset structure.")

# T-Test with Levene's Test for Equal Variance
t_test_results = {'Feature': [], 'T-Statistic': [], 'P-Value': [], 'Equal Variance': []}

# Exclude 'Diagnosis' column from feature list
for feature in df.columns:
    if feature != 'Diagnosis':
        # Check variance equality using Levene’s test
        stat, p = levene(benign[feature], malignant[feature])
        qual_var = p > 0.05

        # Conduct independent t-test
        t_stat, p_value = ttest_ind(benign[feature], malignant[feature], equal_var=equal_var)
        
        # Store results
        t_test_results['Feature'].append(feature)
        t_test_results['T-Statistic'].append(t_stat)
        t_test_results['P-Value'].append(p_value)
        t_test_results['Equal Variance'].append(equal_var)

# Convert results to DataFrame
t_test_df = pd.DataFrame(t_test_results)

#Filter for significant results
significant_features = t_test_df[t_test_df['P-Value'] < 0.05]
print(significant_features)

       'x.smoothness_mean', 'x.compactness_mean', 'x.concavity_mean',
       'x.concave_pts_mean', 'x.symmetry_mean', 'x.fractal_dim_mean',
       'x.radius_se', 'x.texture_se', 'x.perimeter_se', 'x.area_se',
       'x.smoothness_se', 'x.compactness_se', 'x.concavity_se',
       'x.concave_pts_se', 'x.symmetry_se', 'x.fractal_dim_se',
       'x.radius_worst', 'x.texture_worst', 'x.perimeter_worst',
       'x.area_worst', 'x.smoothness_worst', 'x.compactness_worst',
       'x.concavity_worst', 'x.concave_pts_worst', 'x.symmetry_worst',
       'x.fractal_dim_worst', 'Diagnosis'],
      dtype='object')
                Feature  T-Statistic       P-Value  Equal Variance
0         x.radius_mean   -22.208798  1.684459e-64           False
1        x.texture_mean   -11.022087  3.019055e-25           False
2      x.perimeter_mean   -22.935314  1.023141e-66           False
3           x.area_mean   -19.640990  3.284366e-52           False
4     x.smoothness_mean    -9.297355  5.573331e-19        